## Data treatment
### 12/09/25

In [1]:
#### Load libs
import os
from ase.db import connect
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from __future__ import annotations
import os, json, sqlite3, re, math
from functools import reduce
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from tqdm import tqdm 
from scipy.stats import pearsonr 

# --- File Configuration ---
C2DB_PATH = "c2db.db"
MP2D_PATH = "2dmatpedia_final.db"
OUTPUT_DB_PATH = "combined_2d_database.db"

## Combine C2DB and 2DMatPedia

In [ ]:
# --- Helper Functions ---

def get_prototypes_and_data(db_path: str):
    """
    Reads an ASE database and returns two things:
    1. A set with all unique prototypes (formula, sym_type, sym_val).
    2. A dictionary mapping each prototype to the first corresponding 'row' found.
    """
    if not os.path.exists(db_path):
        print(f"WARNING: File not found: {db_path}")
        return set(), {}

    prototypes = set()
    proto_to_row_map = {}
    db = connect(db_path)
    
    # Identify symmetry columns
    try:
        row_sample = next(db.select(limit=1))
        keys = list(row_sample.key_value_pairs.keys()) + list(getattr(row_sample, 'data', {}).keys())
    except StopIteration:
        return set(), {}

    num_col = next((c for c in ["sg_number", "number", "lgnum"] if c in keys), None)
    sym_col = next((c for c in ["international", "layergroup"] if c in keys), None)

    for row in db.select():
        formula = row.get("formula_norm", row.formula)
        prototype = None
        
        if num_col and row.get(num_col) is not None:
            try:
                prototype = (formula, "number", str(int(row.get(num_col))))
            except (ValueError, TypeError): pass
        elif sym_col and row.get(sym_col) is not None:
            try:
                sym_symb = re.sub(r"\\s+", "", str(row.get(sym_col)).strip()).replace('"', '').replace("'", "")
                if sym_symb:
                    prototype = (formula, "symbol", sym_symb)
            except (ValueError, TypeError): pass

        if prototype:
            prototypes.add(prototype)
            if prototype not in proto_to_row_map:
                proto_to_row_map[prototype] = row
                
    print(f"Loaded {len(prototypes)} unique prototypes from '{db_path}'.")
    return prototypes, proto_to_row_map

# --- Main Union Logic ---
if __name__ == "__main__":
    if os.path.exists(OUTPUT_DB_PATH):
        os.remove(OUTPUT_DB_PATH)
        print(f"Old database file '{OUTPUT_DB_PATH}' removed.")

    # 1. Load all data and prototypes from each DB
    c2_prototypes, c2_map = get_prototypes_and_data(C2DB_PATH)
    m2_prototypes, m2_map = get_prototypes_and_data(MP2D_PATH)

    # 2. Identify prototype groups
    common_prototypes = c2_prototypes.intersection(m2_prototypes)
    c2_only_prototypes = c2_prototypes - m2_prototypes
    m2_only_prototypes = m2_prototypes - c2_prototypes

    print("\n--- Prototype Summary ---")
    print(f"C2DB exclusive: {len(c2_only_prototypes)}")
    print(f"2DMatPedia exclusive: {len(m2_only_prototypes)}")
    print(f"Common: {len(common_prototypes)}")

    # 3. Open the new database for writing
    with connect(OUTPUT_DB_PATH) as new_db:
        print(f"\nCreating new database at '{OUTPUT_DB_PATH}'...")

        # Process C2DB exclusive materials (copy as is)
        for proto in c2_only_prototypes:
            row = c2_map[proto]
            new_db.write(row.toatoms(), key_value_pairs=row.key_value_pairs, data=row.get('data'))

        # Process 2DMatPedia exclusive materials (with modifications)
        for proto in m2_only_prototypes:
            row = m2_map[proto]
            atoms = row.toatoms()
            kvp = dict(row.key_value_pairs) 

            # Rename 'bandgap' to 'gap'
            if 'bandgap' in kvp:
                kvp['gap'] = kvp.pop('bandgap')
            
            # Create 'is_magnetic' and then rename 'total_magnetization' to 'magmom'
            if 'total_magnetization' in kvp:
                mag = kvp['total_magnetization']
                kvp['is_magnetic'] = bool(pd.notna(mag) and mag > 0.01)
                # Now rename the original property
                kvp['magmom_u'] = kvp.pop('total_magnetization')

            # Rename 'decomposition_energy' to 'ehull'
            if 'decomposition_energy' in kvp:
                kvp['ehull'] = kvp.pop('decomposition_energy')
            
            new_db.write(atoms, key_value_pairs=kvp, data=row.get('data'))
            
        # Process common materials
        for proto in common_prototypes:
            c2_row = c2_map[proto]
            m2_row = m2_map[proto]

            atoms = c2_row.toatoms()
            kvp = dict(c2_row.key_value_pairs)
            
            m2_kvp = m2_row.key_value_pairs
            for key, value in m2_kvp.items():
                if key != 'bandgap' and key not in kvp:
                    kvp[f"m2_{key}"] = value 

            new_db.write(atoms, key_value_pairs=kvp, data=c2_row.get('data'))

    print("\n--- Process Completed ---")
    final_count = len(connect(OUTPUT_DB_PATH))
    print(f"The new database '{OUTPUT_DB_PATH}' was successfully created and contains {final_count} entries.")

Old database file 'combined_2d_database.db' removed.
Loaded 16504 unique prototypes from 'c2db.db'.
Loaded 6159 unique prototypes from '2dmatpedia_final.db'.

--- Prototype Summary ---
C2DB exclusive: 15284
2DMatPedia exclusive: 4939
Common: 1220

Creating new database at 'combined_2d_database.db'...

--- Process Completed ---
The new database 'combined_2d_database.db' was successfully created and contains 21443 entries.


## Preprocessing, filtering and validation

In [4]:
# --- Configuration ---
INPUT_DB = "combined_2d_database.db"
OUTPUT_DB = "final_processed_database.db"

# --- Filtering and Validation Criteria ---
EHULL_THRESHOLD = 0.1  # eV/atom
MIN_INTERATOMIC_DISTANCE = 0.5  # Angstroms (Å)

def get_stability_value(row):
    """Extracts the stability value, looking for common keys."""
    kvp = row.key_value_pairs
    # Prioritize 'ehull' from C2DB, then look for 2DMatPedia analogs
    if 'ehull' in kvp and kvp['ehull'] is not None:
        return kvp['ehull']
    if 'm2_decomposition_energy' in kvp and kvp['m2_decomposition_energy'] is not None:
        return kvp['m2_decomposition_energy']
    return None # Returns None if no stability key is found

# --- Main Processing Logic ---
if __name__ == "__main__":
    if not os.path.exists(INPUT_DB):
        print(f"ERROR: Input database '{INPUT_DB}' not found.")
        exit()

    if os.path.exists(OUTPUT_DB):
        os.remove(OUTPUT_DB)
        print(f"Old database file '{OUTPUT_DB}' removed.")

    # Connect to databases
    db_in = connect(INPUT_DB)
    db_out = connect(OUTPUT_DB)

    # Set to track already added structures and prevent duplicates
    # We use a string representation of the standardized structure as a "fingerprint"
    seen_structures_fingerprints = set()
    
    # Counters for the final report
    total_read = 0
    removed_missing_props = 0
    removed_instability = 0
    removed_invalid_geom = 0
    removed_duplicates = 0
    written_count = 0
    
    print(f"Starting processing of '{INPUT_DB}'...")
    # The tqdm progress bar is useful for long processes
    for row in tqdm(db_in.select(), total=len(db_in)):
        total_read += 1
        
        # --- STEP 1: PROPERTY FILTERING ---
        kvp = row.key_value_pairs
        if kvp.get('gap') is None or kvp.get('is_magnetic') is None:
            removed_missing_props += 1
            continue
            
        stability_val = get_stability_value(row)
        # Filter only if stability is known and above the threshold
        if stability_val is not None and stability_val > EHULL_THRESHOLD:
            removed_instability += 1
            continue

        # --- STEP 2: GEOMETRY STANDARDIZATION AND VALIDATION ---
        try:
            atoms = row.toatoms()
            # Convert from ASE to Pymatgen to use its advanced tools
            structure_pmg = AseAtomsAdaptor.get_structure(atoms)

            # Standardize the cell to the conventional primitive (includes Niggli reduction)
            sga = SpacegroupAnalyzer(structure_pmg, symprec=0.1)
            standardized_structure = sga.get_primitive_standard_structure()
            
            # Validation: check minimum interatomic distance
            # This check is only meaningful for structures with more than one atom.
            if len(standardized_structure) > 1:
                # Get all non-zero distances. Using the upper triangle (k=1) is efficient.
                distances = standardized_structure.distance_matrix[np.triu_indices(len(standardized_structure), k=1)]
                
                # It's possible for the distances array to be empty if the structure is unusual,
                # so we check before calling min().
                if distances.size > 0:
                    min_dist = np.min(distances)
                    if min_dist < MIN_INTERATOMIC_DISTANCE:
                        removed_invalid_geom += 1
                        continue

        except Exception as e:
            # Skip structures that cause errors during conversion or analysis
            # print(f"Warning: Error processing structure ID {row.id} ({row.formula}): {e}")
            removed_invalid_geom += 1
            continue

        # --- STEP 3: DUPLICATE FILTERING ---
        # Create a "fingerprint" of the standardized structure
        # We use the formula and the string representation of the structure for greater robustness
        fingerprint = (
            standardized_structure.formula, 
            standardized_structure.to(fmt="cif")
        )

        if fingerprint in seen_structures_fingerprints:
            removed_duplicates += 1
            continue
        else:
            seen_structures_fingerprints.add(fingerprint)

        # --- STEP 4: SAVE TO NEW DATABASE ---
        # Convert the standardized structure back to ASE format
        final_atoms = AseAtomsAdaptor.get_atoms(standardized_structure)
        
        # Write the standardized structure with original properties
        db_out.write(final_atoms, key_value_pairs=kvp, data=row.get('data'))
        written_count += 1

    # --- Final Report ---
    print("\n--- Processing Complete ---")
    print(f"Total structures read: {total_read}")
    print("-" * 30)
    print(f"Removed due to missing 'gap' or 'is_magnetic': {removed_missing_props}")
    print(f"Removed due to instability (ehull > {EHULL_THRESHOLD} eV/atom): {removed_instability}")
    print(f"Removed due to invalid geometry (dist < {MIN_INTERATOMIC_DISTANCE} Å): {removed_invalid_geom}")
    print(f"Removed due to being duplicates (after standardization): {removed_duplicates}")
    print("-" * 30)
    print(f"Total structures saved to '{OUTPUT_DB}': {written_count}")

Old database file 'final_processed_database.db' removed.
Starting processing of 'combined_2d_database.db'...


  0%|          | 0/21443 [00:00<?, ?it/s]

100%|██████████| 21443/21443 [01:36<00:00, 222.86it/s]


--- Processing Complete ---
Total structures read: 21443
------------------------------
Removed due to missing 'gap' or 'is_magnetic': 8090
Removed due to instability (ehull > 0.1 eV/atom): 6043
Removed due to invalid geometry (dist < 0.5 Å): 0
Removed due to being duplicates (after standardization): 0
------------------------------
Total structures saved to 'final_processed_database.db': 7310
